# 初始化 

In [2]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")


In [3]:
projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

D:\project_meta\NNproject\NNI
{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_unifor

# 模型

In [4]:
from project.model.CNN1D import *
from torchvision.datasets import *
from torchvision.transforms import *
import torchvision
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
MNISTDataset=FashionMNIST(root=r'D:/dataset',transform=train_transforms)


In [40]:
from typing import Any, Optional
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT
from torchmetrics.functional import accuracy
class Base(pl.LightningModule):
    def __init__(self, net,classes,lossFuncthion,optimizer,lrScheduler,*args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)  
        self.Net=net
        self.flatten=nn.Flatten()
        self.LossFunction=nn.CrossEntropyLoss()
        self.Optimizer=optimizer
        self.classes=classes
        
        # self.LrScheduler=lrScheduler

    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        # print(x.shape)
        x=self.flatten(x)
        x=x.unsqueeze(1)
        # print(x.shape)
        y=self.Net(x)
        return y#torch.rand([1,10])
    def configure_optimizers(self) -> Any:
        return [self.Optimizer]
    def training_step(self, batch,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        
        x,y=batch
        print(x.shape)
        y_hat=self.forward(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=self.classes)
        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("trainloss", loss, on_step=True, on_epoch=True)
        return {'loss':loss}
    def validation_step(self,batch, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
        x,y=batch
        y_hat=self.forward(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=self.classes)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        self.log("testloss", loss, on_step=False, on_epoch=True)
        return {'loss':loss}
        

In [41]:
from torchsummary import summary
from project.model.CNN1D import *
from torchvision.models import resnet50,ResNet50_Weights
net=simple_cnn1d(1,10,[32,16],16)
plnet=Base(net,lossFuncthion='none',classes=10,optimizer=torch.optim.Adam(net.parameters(),lr=0.01),lrScheduler=None)
summary(plnet,(1,28,28),batch_size=2,device='cpu')

torch.Size([2, 1, 28, 28])
torch.Size([2, 1, 784])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                   [2, 784]               0
            Conv1d-2               [2, 32, 784]             256
       BatchNorm1d-3               [2, 32, 784]              64
              ReLU-4               [2, 32, 784]               0
         MaxPool1d-5               [2, 32, 391]               0
            Conv1d-6               [2, 16, 196]           1,552
       BatchNorm1d-7               [2, 16, 196]              32
              ReLU-8               [2, 16, 196]               0
 AdaptiveAvgPool1d-9                 [2, 16, 1]               0
          Flatten-10                    [2, 16]               0
             ReLU-11                    [2, 16]               0
           Linear-12                    [2, 16]             272
             ReLU-13                    [2, 16]     

In [38]:
torch.rand([2,748]).unsqueeze(1).shape

torch.Size([2, 1, 748])

In [42]:
from torch.utils.data import random_split
train, val = random_split(MNISTDataset, [55000, 5000])
trainer = pl.Trainer()
trainer.progress_bar_refresh_rate = 10
trainer.fit(plnet, DataLoader(train,batch_size=64),DataLoader(val,batch_size=64))#,

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | Net          | simple_cnn1d     | 2.3 K 
1 | flatten      | Flatten          | 0     
2 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
2.3 K     Trainable params
0         Non-trainable params
2.3 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])


g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1,

Validation: 0it [00:00, ?it/s]

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1,

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
